<a href="https://colab.research.google.com/github/KoreanAImaster/Tensorflow/blob/main/%EB%A1%9C%EB%B4%87%EC%8B%AC%ED%8C%90AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

FoulOrNot = "/tmp/FoulOrNot"

try:
    os.mkdir('/tmp/RefereeAI')
    os.mkdir('/tmp/RefereeAI/training')
    os.mkdir('/tmp/RefereeAI/testing')
    os.mkdir('/tmp/RefereeAI/training/foul')
    os.mkdir('/tmp/RefereeAI/training/nofoul')
    os.mkdir('/tmp/RefereeAI/testing/foul')
    os.mkdir('/tmp/RefereeAI/testing/nofoul')
except OSError:
    pass

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)

FOUL_SOURCE_DIR = "/tmp/FoulOrNot/Foul/"
TRAINING_FOUL_DIR = "/tmp/RefereeAI/training/foul/"
TESTING_FOUL_DIR = "/tmp/RefereeAI/testing/foul/"
NOFOUL_SOURCE_DIR = "/tmp/FoulOrNot/NoFoul/"
TRAINING_NOFOUL_DIR = "/tmp/RefereeAI/training/nofoul/"
TESTING_NOFOUL_DIR = "/tmp/RefereeAI/testing/nofoul/"

split_size = .8
split_data(FOUL_SOURCE_DIR, TRAINING_FOUL_DIR, TESTING_FOUL_DIR, split_size)
split_data(NOFOUL_SOURCE_DIR, TRAINING_NOFOUL_DIR, TESTING_NOFOUL_DIR, split_size)

print(len(os.listdir('/tmp/RefereeAI/training/foul/')))
print(len(os.listdir('/tmp/RefereeAI/training/nofoul/')))
print(len(os.listdir('/tmp/RefereeAI/testing/foul/')))
print(len(os.listdir('/tmp/RefereeAI/testing/nofoul/')))


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

TRAINING_DIR = "/tmp/RefereeAI/training/"
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=10,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

VALIDATION_DIR = "/tmp/RefereeAI/testing/"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=10,
                                                              class_mode='binary',
                                                              target_size=(150, 150))
history = model.fit(train_generator,
                              epochs=50,
                              verbose=1,
                              validation_data=validation_generator)

15
15
10
10
Found 30 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/50
3/3 [==============================] - 2s 484ms/step - loss: 2.8818 - accuracy: 0.3667 - val_loss: 0.6619 - val_accuracy: 0.6500
Epoch 2/50
3/3 [==============================] - 1s 372ms/step - loss: 0.6225 - accuracy: 0.7333 - val_loss: 0.4363 - val_accuracy: 0.9500
Epoch 3/50
3/3 [==============================] - 1s 367ms/step - loss: 0.9780 - accuracy: 0.6333 - val_loss: 0.5288 - val_accuracy: 0.8000
Epoch 4/50
3/3 [==============================] - 1s 363ms/step - loss: 0.5594 - accuracy: 0.6333 - val_loss: 0.3884 - val_accuracy: 0.9500
Epoch 5/50
3/3 [==============================] - 1s 368ms/step - loss: 0.3612 - accuracy: 0.8667 - val_loss: 0.2245 - val_accuracy: 0.9500
Epoch 6/50
3/3 [==============================] - 1s 382ms/step - loss: 0.1734 - accuracy: 1.0000 - val_loss: 0.0539 - val_accuracy: 1.0000
Epoch 7/50
3/3 [==============================] - 1s 367ms/step - lo

In [ ]:
# Here's a codeblock just for fun. You should be able to upload an image here 
# and have it classified without crashing
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is not foul")
  else:
    print(fn + " is foul")

Saving 201703231534504171_1.jpg to 201703231534504171_1.jpg
[1.]
201703231534504171_1.jpg is not foul
